<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1


    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance


    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-18 13:17:23
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  6.17 L
Current:  1.42 C
-------------------
Today PnL: -12.26 K (-0.09%)
Current PnL: -17.16 L (-11.55%)
CY Booked + Current PnL: -5.72 L (-3.85%)
-------------------
Total profit:  1.99 L
Total loss:  -19.15 L
-------------------
Total Booked + Current PnL: 21.26 L (17.47%)
Total Booked PnL: 38.42 L (31.57%)
Curr Year Booked PnL: 11.44 L (8.03%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.25 C
Est FTT PnL: 82.53 L (57.94%)
Deployed:  1.22 C
Current:  1.42 C
CAGR/XIRR %: 9.75%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TTKPRESTIG,770.00,103.42,54.0,M-SC,3.98,87652.0,-13125.0,13218.0,-1.13,-13.02,15.08,0.09,245.0,-0.99,0.64,15.16,OX40N,NTT,DURABLES
56,RELIANCE,1533.00,-11.79,59.0,H-LC,5.18,220412.0,10046.0,18735.0,-0.06,4.78,8.50,13.68,37.0,0.54,1.62,22.09,XY25,NTT,REFINERIES
49,NATIONALUM,247.44,-43.66,62.0,H-MC,6.78,112122.0,11618.0,19027.0,-0.80,11.56,16.97,30.49,79.0,0.61,0.82,47.71,MH,ATH,METALS
41,ITC,452.00,-37.92,51.0,H-LC,2.53,201052.0,914.0,19985.0,0.45,0.46,9.94,10.44,4.0,0.05,1.48,6.05,X40,NTT,FMCG
48,MASFIN,398.61,-16.89,52.0,H-SC,12.99,94410.0,-3570.0,25170.0,-0.52,-3.64,26.66,22.05,152.0,-0.14,0.69,37.23,XR,ATH,FINANCE


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,SAIL,228.00,43.54,61.0,M-MC,6.95,226684.0,1722.0,165933.0,-0.84,0.77,73.20,74.53,192.0,0.01,1.66,33.27,XY24,BTT,STEEL
35,IEX,219.00,-29.69,55.0,H-SC,17.08,200904.0,950.0,96273.0,-0.26,0.48,47.92,48.63,115.0,0.01,1.47,12.08,XR,NTT,MISC
34,ICICIPRULI,790.00,-20.54,46.0,H-MC,2.23,136640.0,852.0,41894.0,0.75,0.63,30.66,31.48,107.0,0.02,1.00,12.82,X40,ATH,INSURANCE
29,HAVELLS,2069.16,-0.23,56.0,H-MC,1.86,247463.0,-268.0,75328.0,-0.45,-0.11,30.44,30.30,92.0,-0.00,1.82,13.17,X40,ATH,ELECTRICAL
51,PGHH,17973.08,-30.68,53.0,H-MC,4.20,200700.0,-120.0,68900.0,-0.03,-0.06,34.33,34.25,80.0,-0.00,1.47,5.13,X40,ATH,FMCG


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VBL,671.64,-17.48,38.0,H-LC,6.40,297340.0,-18502.0,131157.0,-1.06,-5.86,44.11,35.67,5.0,-0.14,2.18,7.18,X40N,ATH,FMCG
5,ASIANPAINT,3465.66,-13.45,45.0,H-LC,7.81,218539.0,-33229.0,86432.0,-0.41,-13.20,39.55,21.13,27.0,-0.38,1.60,17.55,X40,ATH,PAINTS
79,UNITDSPR,1644.00,-13.37,50.0,H-LC,7.82,227143.0,-7001.0,55627.0,-1.23,-2.99,24.49,20.77,86.0,-0.13,1.67,3.72,X40N,NTT,BREWERIES
41,ITC,452.00,-37.92,51.0,H-LC,2.53,201052.0,914.0,19985.0,0.45,0.46,9.94,10.44,4.0,0.05,1.48,6.05,X40,NTT,FMCG
43,JIOFIN,387.00,0.15,51.0,H-LC,13.22,272104.0,4739.0,60706.0,0.17,1.77,22.31,24.48,48.0,0.08,2.00,57.72,XY24,BTT,FINANCE


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,LTIM,7230.2,0.87,72.0,H-LC,1.29,250042.0,3891.0,75313.0,2.46,1.58,30.12,32.18,16.0,0.05,1.83,38.19,X200,ATH,IT
41,ITC,452.0,-37.92,51.0,H-LC,2.53,201052.0,914.0,19985.0,0.45,0.46,9.94,10.44,4.0,0.05,1.48,6.05,X40,NTT,FMCG
50,NESTLEIND,1377.0,-8.22,57.0,H-LC,3.85,282251.0,16825.0,39967.0,0.16,6.34,14.16,21.40,11.0,0.42,2.07,13.40,XY25,NTT,FMCG
56,RELIANCE,1533.0,-11.79,59.0,H-LC,5.18,220412.0,10046.0,18735.0,-0.06,4.78,8.50,13.68,37.0,0.54,1.62,22.09,XY25,NTT,REFINERIES
19,CIPLA,1795.0,-19.44,54.0,H-LC,5.78,214953.0,10453.0,30953.0,0.60,5.11,14.40,20.25,10.0,0.34,1.58,14.29,X40N,BTT,PHARMA


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,LTIM,7230.20,0.87,72.0,H-LC,1.29,250042.0,3891.0,75313.0,2.46,1.58,30.12,32.18,16.0,0.05,1.83,38.19,X200,ATH,IT
60,SAMMAANCAP,326.00,-171.59,64.0,M-SC,2.64,82764.0,-19446.0,112832.0,2.12,-19.03,136.33,91.37,208.0,-0.17,0.61,34.35,XY25,NTT,FINANCE
80,VAIBHAVGBL,521.00,66.94,72.0,H-SC,7.21,144954.0,-37821.0,148897.0,1.51,-20.69,102.72,60.77,125.0,-0.25,1.06,29.83,XR,NTT,JEWELLERY
46,LAOPALA,464.00,115.96,52.0,H-SC,2.37,98781.0,-32324.0,83569.0,1.50,-24.65,84.60,39.09,142.0,-0.39,0.72,27.22,AR,NTT,CERAMICS
17,CAMS,5211.76,0.38,66.0,H-SC,3.81,114010.0,12006.0,31923.0,1.45,11.77,28.00,43.06,122.0,0.38,0.84,30.90,X40N,ATH,MISC


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,GREENPANEL,537.00,229.46,60.0,M-SC,2.49,147230.0,-35848.0,114825.0,-2.66,-19.58,77.99,43.14,230.0,-0.31,1.08,38.49,XY24,NTT,MISC
20,COALINDIA,497.97,8.96,58.0,L-LC,2.35,145003.0,-5458.0,39252.0,-1.96,-3.63,27.07,22.46,167.0,-0.14,1.06,13.80,XY25,ATH,MINING
54,RAJOOENG,143.33,-36.44,37.0,H-SC,11.61,96220.0,-6280.0,47109.0,-1.82,-6.13,48.96,39.83,114.0,-0.13,0.71,0.57,AR,ATH,MISC
68,SURYODAY,240.00,44.74,57.0,H-SC,2.51,133314.0,-45757.0,111490.0,-1.68,-25.55,83.63,36.71,135.0,-0.41,0.98,32.27,XR,NTT,BANKS
61,SATIN,274.00,-27.73,50.0,H-SC,5.90,171174.0,-54184.0,152413.0,-1.67,-24.04,89.04,43.59,126.0,-0.36,1.26,8.10,XY24,NTT,FINANCE


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TTKPRESTIG,770.0,103.42,54.0,M-SC,3.98,87652.0,-13125.0,13218.0,-1.13,-13.02,15.08,0.09,245.0,-0.99,0.64,15.16,OX40N,NTT,DURABLES
77,UJJIVANSFB,60.0,94.87,54.0,H-SC,9.52,123390.0,-19089.0,38609.0,-0.11,-13.40,31.29,13.70,163.0,-0.49,0.91,45.63,OX40N,NTT,BANKS
44,KANSAINER,340.0,-67.06,63.0,H-SC,1.41,226809.0,-42858.0,79202.0,1.01,-15.89,34.92,13.47,138.0,-0.54,1.66,14.72,XY24,NTT,PAINTS
18,CERA,9475.0,-19.33,50.0,H-SC,2.08,146890.0,-29013.0,71036.0,0.20,-16.49,48.36,23.89,149.0,-0.41,1.08,26.86,OX40N,NTT,CERAMICS
71,TATAELXSI,9161.0,-19.68,53.0,H-MC,7.67,108585.0,-23250.0,65477.0,0.56,-17.64,60.30,32.03,98.0,-0.36,0.80,21.62,OX40N,NTT,IT


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4476.75,-25.15,59.0,H-LC,12.34,297839.0,-48117.0,122978.0,-0.14,-13.91,41.29,21.64,1.0,-0.39,2.19,5.51,X40,ATH,IT
39,INFY,2275.00,-15.65,59.0,H-LC,8.41,328659.0,15771.0,155916.0,1.35,5.04,47.44,54.87,3.0,0.10,2.41,12.03,X40,BTT,IT
41,ITC,452.00,-37.92,51.0,H-LC,2.53,201052.0,914.0,19985.0,0.45,0.46,9.94,10.44,4.0,0.05,1.48,6.05,X40,NTT,FMCG
82,VBL,671.64,-17.48,38.0,H-LC,6.40,297340.0,-18502.0,131157.0,-1.06,-5.86,44.11,35.67,5.0,-0.14,2.18,7.18,X40N,ATH,FMCG
1,ABB,7934.00,-37.92,66.0,H-LC,10.36,259354.0,-2265.0,121481.0,0.32,-0.87,46.84,45.57,7.0,-0.02,1.90,10.36,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,SAIL,228.00,43.54,61.0,M-MC,6.95,226684.0,1722.0,165933.0,-0.84,0.77,73.20,74.53,192.0,0.01,1.66,33.27,XY24,BTT,STEEL
14,BLUESTARCO,2326.38,13.46,64.0,H-MC,3.56,195910.0,31240.0,36733.0,0.97,18.97,18.75,41.28,89.0,0.85,1.44,28.47,X40N,ATH,AC
83,VOLTAS,1918.49,1.58,60.0,H-MC,2.50,213105.0,21363.0,74672.0,0.62,11.14,35.04,50.08,99.0,0.29,1.56,18.70,XY25,ATH,AC
17,CAMS,5211.76,0.38,66.0,H-SC,3.81,114010.0,12006.0,31923.0,1.45,11.77,28.00,43.06,122.0,0.38,0.84,30.90,X40N,ATH,MISC
23,DABUR,735.00,-4.38,52.0,H-MC,6.71,208221.0,11193.0,78437.0,-0.23,5.68,37.67,45.49,102.0,0.14,1.53,19.46,XY24,BTT,FMCG


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,NATIONALUM,247.44,-43.66,62.0,H-MC,6.78,112122.0,11618.0,19027.0,-0.80,11.56,16.97,30.49,79.0,0.61,0.82,47.71,MH,ATH,METALS
84,WHIRLPOOL,2270.00,-40.92,50.0,M-SC,0.47,99930.0,8432.0,70321.0,-0.69,9.21,70.37,86.07,223.0,0.12,0.73,45.12,XR,NTT,DURABLES
36,INDIAMART,4850.92,-52.21,40.0,H-SC,11.64,133571.0,10235.0,123526.0,-1.20,8.30,92.48,108.45,119.0,0.08,0.98,32.51,AR,ATH,MISC
85,WIPRO,420.00,-9.71,60.0,M-LC,5.63,159061.0,8116.0,101338.0,0.94,5.38,63.71,72.51,53.0,0.08,1.17,11.72,XR,NTT,IT
11,BANKINDIA,190.00,-29.19,60.0,H-MC,6.07,181945.0,2137.0,110277.0,-1.12,1.19,60.61,62.52,88.0,0.02,1.34,35.68,XR,NTT,BANKS


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,6744.44,54.0,L-SC,5.19,75075.0,-18735.0,95353.0,1.33,-19.97,127.01,81.67,269.0,-0.20,0.55,45.39,XR,NTT,CERAMICS
60,SAMMAANCAP,326.00,-171.59,64.0,M-SC,2.64,82764.0,-19446.0,112832.0,2.12,-19.03,136.33,91.37,208.0,-0.17,0.61,34.35,XY25,NTT,FINANCE
21,COFFEEDAY,80.00,-41.65,52.0,L-SC,42.32,84960.0,-28589.0,68639.0,-0.18,-25.18,80.79,35.27,268.0,-0.42,0.62,107.94,XR,NTT,HOTELS
3,ALKYLAMINE,4546.37,-4.34,48.0,H-SC,12.74,89092.0,-11871.0,106403.0,-1.05,-11.76,119.43,93.63,148.0,-0.11,0.65,32.70,SR,ATH,CHEMICALS
48,MASFIN,398.61,-16.89,52.0,H-SC,12.99,94410.0,-3570.0,25170.0,-0.52,-3.64,26.66,22.05,152.0,-0.14,0.69,37.23,XR,ATH,FINANCE


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ATULAUTO,844.0,3900.00,64.0,M-SC,5.82,124738.0,-22630.0,74444.0,0.94,-15.36,59.68,35.16,236.0,-0.30,0.92,27.41,XY24,NTT,AUTO
60,SAMMAANCAP,326.0,-171.59,64.0,M-SC,2.64,82764.0,-19446.0,112832.0,2.12,-19.03,136.33,91.37,208.0,-0.17,0.61,34.35,XY25,NTT,FINANCE
21,COFFEEDAY,80.0,-41.65,52.0,L-SC,42.32,84960.0,-28589.0,68639.0,-0.18,-25.18,80.79,35.27,268.0,-0.42,0.62,107.94,XR,NTT,HOTELS
80,VAIBHAVGBL,521.0,66.94,72.0,H-SC,7.21,144954.0,-37821.0,148897.0,1.51,-20.69,102.72,60.77,125.0,-0.25,1.06,29.83,XR,NTT,JEWELLERY
74,TITAGARH,1548.0,0.74,68.0,H-SC,3.27,165104.0,-26911.0,105799.0,0.13,-14.02,64.08,41.08,158.0,-0.25,1.21,39.91,XY24,NTT,ENGINEERING


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.12
1,25,43.92
2,50,74.47


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.53
LC    32.18
MC    23.30
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.25
X40      15.47
X40N     11.96
XR       11.76
AR        8.76
XY25      8.27
OX40N     8.25
X200      1.83
MH        1.71
X5K       1.45
SR        1.30
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.31
H-LC    25.47
H-MC    20.34
M-SC    13.65
M-LC     5.65
M-MC     2.65
L-SC     1.57
L-LC     1.06
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
IT,13.73,-12.48,73.81
FMCG,11.90,-4.41,40.94
FINANCE,9.75,-15.85,61.22
MISC,7.49,-12.51,75.08
BANKS,6.30,-16.46,77.73
PAINTS,5.87,-15.31,32.49
ELECTRICAL,5.77,-4.85,62.65
AC,3.72,4.49,27.86
INSURANCE,3.43,-7.37,44.20


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2858037.0,22
XR,1306414.0,14
AR,1303860.0,9
X40,741193.0,10
X40N,601684.0,9
OX40N,523991.0,10
XY25,395371.0,6
SR,244254.0,2
X5K,120487.0,1


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3417554.0,29
M-SC,1385056.0,17
H-LC,1223104.0,14
H-MC,1183218.0,15
M-LC,381083.0,4
M-MC,315569.0,2
L-SC,247133.0,3
L-MC,60796.0,1
L-LC,39252.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1031561.0      6
M-SC       XY24       799335.0      7
H-SC       AR         788661.0      5
           XR         754537.0      7
H-MC       XY24       491608.0      4
H-LC       X40        456585.0      5
           AR         354061.0      2
M-MC       XY24       315569.0      2
H-SC       X40N       286014.0      4
           OX40N      249393.0      4
           SR         244254.0      2
H-LC       X40N       217737.0      3
H-MC       X40        214311.0      4
           XY25       184585.0      2
L-SC       XR         163992.0      2
M-SC       AR         161138.0      2
M-LC       XY24       159258.0      2
M-SC       OX40N      125980.0      4
M-LC       X5K        120487.0      1
M-SC       XR         115474.0      2
           XY25       112832.0      1
H-MC       XR         110277.0      1
M-LC       XR         101338.0      1
H-MC       X40N        97933.0      2
L-SC       OX40N       83141.0      1
H-LC       X200        75313.0      1
M-SC       X40         70297.0      1
H-MC       OX40N       65477.0      1
H-SC       MH          63134.0      1
L-MC       XR          60796.0      1
H-LC       XY24        60706.0      1
           XY25        58702.0      2
L-LC       XY25        39252.0      1
H-MC       MH          19027.0      1

In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
